# Text summarization with small files with Amazon Titan

> *This notebook should work well with the **`Data Science 2.0`** kernel in SageMaker Studio*

## Overview

In this example, you are going to ingest a small amount of data (String data) directly into Amazon Bedrock API (using Amazon Titan model) and give it an instruction to summarize the respective text.

### Architecture

![](./images/41-text-simple-1.png)

In this architecture:

1. A small piece of text (or small file) is loaded
1. A foundational model processes those data
1. Model returns a response with the summary of the ingested text

### Use case

This approach can be used to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

### Challenges
This approach can be used when the input text or file fits within the model context length. In notebook `02.long-text-summarization-titan.ipynb`, we will explore an approach to address the challenge when users have large document(s) that exceed the token limit.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [2]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --force-reinstall \
    ../dependencies/awscli-1.27.162-py3-none-any.whl \
    ../dependencies/boto3-1.26.162-py3-none-any.whl \
    ../dependencies/botocore-1.29.162-py3-none-any.whl

%pip install --quiet langchain==0.0.249

Processing /root/amazon-bedrock-workshop/dependencies/awscli-1.27.162-py3-none-any.whl
Processing /root/amazon-bedrock-workshop/dependencies/boto3-1.26.162-py3-none-any.whl
Processing /root/amazon-bedrock-workshop/dependencies/botocore-1.29.162-py3-none-any.whl
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached s3transfer-0.6.1-py3-none-any.whl (79 kB)
  Using cached PyYAML-5.4.1-cp38-cp38-manylinux1_x86_64.whl (662 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.16-py2.py3-none-any.whl.metadata (48 kB)
  Using cached six-1.16.0-py2.py3-

In [3]:
import json
import os
import sys

import boto3
import yaml

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


with open('../config.yml', 'r') as file:
    config = yaml.safe_load(file)

b_endpoint = config['bedrock-preview']['endpoint']
b_region = config['bedrock-preview']['region']


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=b_endpoint,
    region=b_region,
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock(https://prod.us-west-2.frontend.bedrock.aws.dev)


## Summarizing a short text with boto3
 
To learn detail of API request to Amazon Bedrock, this notebook introduces how to create API request and send the request via Boto3 rather than relying on langchain, which gives simpler API by wrapping Boto3 operation. 

### Request Syntax of InvokeModel in Boto3


We use `InvokeModel` API for sending request to a foundation model. Here is an example of API request for sending text to Amazon Titan Text Large. Inference parameters in `textGenerationConfig` depends on the model that you are about to use. Inference paramerters of Amazon Titan Text are:
- **maxTokenCount** configures the max number of tokens to use in the generated response. (int, defaults to 512)
- **stopSequences** is used to make the model stop at a desired point, such as the end of a sentence or a list. The returned response will not contain the stop sequence.
- **temperature** modulates the probability density function for the next tokens, implementing the temperature sampling technique. This parameter can be used to deepen or flatten the density function curve. A lower value results in a steeper curve and more deterministic responses, whereas a higher value results in a flatter curve and more random responses. (float, defaults to 0, max value is 1.5)
- **topP** controls token choices, based on the probability of the potential choices. If you set Top P below 1.0, the model considers only the most probable options and ignores less probable options. The result is more stable and repetitive completions.

```python
response = bedrock.invoke_model(body={
                                   "inputText": "this is where you place your input text",
                                   "textGenerationConfig": {
                                       "maxTokenCount": 4096,
                                       "stopSequences": [],
                                       "temperature":0,
                                       "topP":1
                                       },
                                },
                                modelId="amazon.titan-tg1-large", 
                                accept=accept, 
                                contentType=contentType)

```

### Writing prompt with text to be summarized

In this notebook, you can use any short text whose tokens are less than the maximum token of a foundation model. As an exmple of short text, let's take one paragraph of an [AWS blog post](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) about announcement of Amazon Bedrock.

The prompt starts with an instruction `Please provide a summary of the following text.`, and includes text surrounded by  `<text>` tag. 

In [4]:
prompt = """
Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.

<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

"""

## Creating request body with prompt and inference parameters 

Following the request syntax of `invoke_model`, you create request body with the above prompt and inference parameters.

In [5]:
body = json.dumps({"inputText": prompt, 
                   "textGenerationConfig":{
                       "maxTokenCount":4096,
                       "stopSequences":[],
                       "temperature":0,
                       "topP":1
                   },
                  }) 

## Invoke foundation model via Boto3

Here sends the API request to Amazon Bedrock with specifying request parameters `modelId`, `accept`, and `contentType`. Following the prompt, the foundation model in Amazon Bedrock sumamrizes the text.

In [6]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('results')[0].get('outputText'))

Amazon Bedrock is a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon
accessible via an API. It offers the ability to access a range of powerful FMs for text and images
through a scalable, reliable, and secure AWS managed service. Customers can easily find the right
model for what they're trying to get done, get started quickly, privately customize FMs with their
own data, and easily integrate and deploy them into their applications using the AWS tools and
capabilities they are familiar with.


In the above the Bedrock service generates the entire summary for the given prompt in a single output, this can be slow if the output contains large amount of tokens. 

Below we explore the option how we can use Bedrock to stream the output such that the user could start consuming it as it is being generated by the model. For this Bedrock supports `invoke_model_with_response_stream` API providing `ResponseStream` that streams the output in form of chunks.

In [7]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"outputText":"Amazon Bedrock is a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. It offers the ability to access a range of powerful FMs for text and imag","index":0,"totalOutputTextTokenCount":null,"completionReason":null,"inputTextTokenCount":255}'}},
 {'chunk': {'bytes': b'{"outputText":"es through a scalable, reliable, and secure AWS managed service. Customers can easily find the right model for what they\'re trying to get done, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS tools and capabilities they are familiar with.","index":0,"totalOutputTextTokenCount":108,"completionReason":"FINISH","inputTextTokenCount":null}'}}]

Instead of generating the entire output, Bedrock sends smaller chunks from the model. This can be displayed in a consumable manner as well.

In [8]:
from IPython.display import display_markdown, Markdown, clear_output

In [10]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

clear_output(wait=True)
print_ww(''.join(output))

Amazon Bedrock is a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon
accessible via an API. It offers the ability to access a range of powerful FMs for text and images
through a scalable, reliable, and secure AWS managed service. Customers can easily find the right
model for what they're trying to get done, get started quickly, privately customize FMs with their
own data, and easily integrate and deploy them into their applications using the AWS tools and
capabilities they are familiar with.


## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating a summary of AWS news about Amazon Bedrock in 2 different ways: entire output and streaming output generation.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You